# [Module 1.2] AFD TRANSACTION_FRAUD_INSIGHTS 모델 타입 데이터 준비

이 노트북은 주로 아래와 같은 작업을 합니다.
- 데이터 세트 로딩
- AFD TRANSACTION_FRAUD_INSIGHTS 모델 타입 데이터 준비

---

### 참고 자료
- AFD TRANSACTION_FRAUD_INSIGHTS 모델 타입은 신규 기능 입니다. 자세한 사항은 아래 개발자 가이드를 꼭 참조 하시기 바랍니다.
- 신규 모델 타입 --> [Transaction fraud insights](https://docs.aws.amazon.com/frauddetector/latest/ug/transaction-fraud-insights.html)

# 1. 환경 셋업
- 아래는 파이썬 캐키지를 임포트할때에 캐싱된 것을 사용하지 않고, 매번 리로딩 하는 세팅 입니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%store -r train_local_path
%store -r test_local_path

In [3]:
import pandas as pd
import os


# 2. 데이터 세트 로딩 


In [4]:
train_df = pd.read_csv(train_local_path)
test_df = pd.read_csv(test_local_path)
train_df

,EVENT_LABEL,str_ip,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP
0,0,str_62184,str_13,str_1,str_19,str_477,2017-11-07 08:48:38
1,0,str_117038,str_11,str_1,str_19,str_173,2017-11-07 08:05:49
2,0,str_111102,str_27,str_1,str_13,str_153,2017-11-08 02:02:42
3,0,str_118315,str_3,str_1,str_19,str_205,2017-11-07 12:05:08
4,0,str_42280,str_32,str_1,str_13,str_376,2017-11-07 05:08:52
...,...,...,...,...,...,...,...
179995,0,str_4476,str_22,str_1,str_25,str_116,2017-11-08 04:03:42
179996,0,str_79857,str_9,str_1,str_13,str_466,2017-11-08 15:45:35
179997,0,str_45745,str_23,str_1,str_19,str_153,2017-11-07 14:03:17
179998,0,str_44744,str_18,str_1,str_13,str_439,2017-11-08 21:04:59


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180000 entries, 0 to 179999
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   EVENT_LABEL      180000 non-null  int64 
 1   str_ip           180000 non-null  object
 2   str_app          180000 non-null  object
 3   str_device       180000 non-null  object
 4   str_os           180000 non-null  object
 5   str_channel      180000 non-null  object
 6   EVENT_TIMESTAMP  180000 non-null  object
dtypes: int64(1), object(6)
memory usage: 9.6+ MB


# 3. Transaction fraud insights 모델 타입을 위한 필요 컬럼 생성

## 아래 컬럼 추가 생성 필요
- EVENT_ID
    - 각 이벤트에 고유한 카 값을 생성
- ENTITY_TYPE    
    - 실행 주체를 customer 로 지정
- ENTITY_ID
    - 개념적으로 ENTITY_TYPE에 종속이 되고, customer 의 고유 키 값을 의미함. 보통 user_id, email_address 가 됨.
    - 여기의 데이터 셋에는 이러한 데이터가 없기에 str_ip 를 ENTITY_ID 변경 함.
- LABEL_TIMESTAMP
    - 레이블이 업데이트 될 수 있으므로 레이블용 timestamp 를 정의함. 변화가 없으면 EVENT_TIMESTAMP 와 동일하게 함.

## isostandard 8601 UTC 포맷 필요
- LABEL_TIMESTAMP, EVENT_TIMESTAMP

## 참고
- 모델 훈련시에 날짜가 최근 보다 18개월 이전이면 아래와 같은 에러가 발생함.
- 이런 이유로 `pd.DateOffset(years=3)) # 3년 추가`  구문 삽입
```
ValidationException: An error occurred (ValidationException) when calling the CreateModelVersion operation: The StartTime and EndTime cannot be more than 18 months in the past.
```

In [6]:
import uuid

def transform_afd_columns(df, entity_type):
    '''
    Transaction fraud insights 모델 타입을 위한 데이터 컬럼 생성 및 변경
    '''
    ldf = df.copy()
    
    # EVENT_ID 생성
    col_length = len(ldf.columns)
    event_id_list = df['EVENT_TIMESTAMP'].apply(lambda x :  uuid.uuid4())        
    ldf.insert(col_length,column='EVENT_ID', value = event_id_list)

    # ENTITY_TYPE 생성
    ldf.insert(col_length,column='ENTITY_TYPE', value = entity_type)        
    
    # str_ip 를 ENTITY_ID 로 이름 변경
    ldf = ldf.rename(columns={'str_ip':'ENTITY_ID'})    

    # timestamp 를 isostandard 8601 UTC 로 수정    
    ldf['EVENT_TIMESTAMP'] = pd.to_datetime(ldf.EVENT_TIMESTAMP, format='%Y-%m-%dT%H:%M:%S') # + relativedelta(years=1)
    ldf['EVENT_TIMESTAMP'] = ldf['EVENT_TIMESTAMP'].apply(lambda x: x + pd.DateOffset(years=3)) # 3년 추가
    
    ldf['LABEL_TIMESTAMP'] = ldf['EVENT_TIMESTAMP'].apply(lambda x : x.strftime('%Y-%m-%dT%H:%M:%SZ'))    
    ldf['EVENT_TIMESTAMP'] = ldf['EVENT_TIMESTAMP'].apply(lambda x : x.strftime('%Y-%m-%dT%H:%M:%SZ'))        


    
    return ldf

train_df = transform_afd_columns(train_df, entity_type='customer')
test_df = transform_afd_columns(test_df, entity_type='customer')
train_df.head()


,EVENT_LABEL,ENTITY_ID,str_app,str_device,str_os,str_channel,EVENT_TIMESTAMP,ENTITY_TYPE,EVENT_ID,LABEL_TIMESTAMP
0,0,str_62184,str_13,str_1,str_19,str_477,2020-11-07T08:48:38Z,customer,83d3a420-d0b8-4370-804a-95f08964796e,2020-11-07T08:48:38Z
1,0,str_117038,str_11,str_1,str_19,str_173,2020-11-07T08:05:49Z,customer,f83f639e-d377-42e5-aaff-6979ac04f05c,2020-11-07T08:05:49Z
2,0,str_111102,str_27,str_1,str_13,str_153,2020-11-08T02:02:42Z,customer,0bca4042-4f4d-4304-a700-a23a29261a22,2020-11-08T02:02:42Z
3,0,str_118315,str_3,str_1,str_19,str_205,2020-11-07T12:05:08Z,customer,2ce79c2a-afd0-47cd-85dd-a3111d11367c,2020-11-07T12:05:08Z
4,0,str_42280,str_32,str_1,str_13,str_376,2020-11-07T05:08:52Z,customer,e7c5a1e3-b9d6-462f-8f83-9ce3236e1996,2020-11-07T05:08:52Z


# 4. 데이터 저장

## 로컬에 데이터 CSV 로 저장
- 로컬에 훈련 및 테스트 데이터 세트 저장
    - data/train/
    - data/test/

In [7]:
from src.p_utils import save_csv_local

train_file_name = 'train-afd-tran-' + str(train_df.shape[0]) + ".csv"
train_afd_tran_local_path = save_csv_local(raw_df=train_df, preproc_folder='data/train', 
                                  label='EVENT_LABEL', file_name=train_file_name)
print("train_local_path: ", train_local_path)

test_file_name = 'test-afd-tran-' + str(test_df.shape[0]) + ".csv"
test_afd_tran_local_path = save_csv_local(raw_df=test_df, preproc_folder='data/test', 
                                  label='EVENT_LABEL', file_name=test_file_name)
print("test_local_path: ", test_local_path)

data/train/train-afd-tran-180000.csv is saved
train_local_path:  data/train/train-180000.csv
data/test/test-afd-tran-20000.csv is saved
test_local_path:  data/test/test-20000.csv


## S3에 데이로 업로딩
- 위의 파일을 S3에 업로드

In [8]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
# 프로젝트 변수
project_prefix = 'adtalking_fraud_phase0'


# S3에 저장되는 데이터의 기본 폴더 위치
s3_train_afd_tran_data_uri = f"s3://{bucket}/{project_prefix}/train"
s3_test_afd_tran_data_uri = f"s3://{bucket}/{project_prefix}/test"


In [9]:
s3_train_afd_tran_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_afd_tran_local_path, 
    desired_s3_uri=s3_train_afd_tran_data_uri,    
)
print("s3_train_afd_tran_data_uri: \n", s3_train_afd_tran_data_uri)

s3_test_afd_tran_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_afd_tran_local_path, 
    desired_s3_uri=s3_test_afd_tran_data_uri,    
)
print("s3_test_afd_tran_data_uri: \n", s3_test_afd_tran_data_uri)



s3_train_afd_tran_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/train/train-afd-tran-180000.csv
s3_test_afd_tran_data_uri: 
 s3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/test/test-afd-tran-20000.csv


# 5. 변수 저장
- 다른 노트북에서 아래 변수의 값을 사용하기 위해서 저장 합니다.

In [10]:
%store project_prefix
%store bucket
%store train_afd_tran_local_path
%store test_afd_tran_local_path
%store s3_train_afd_tran_data_uri
%store s3_test_afd_tran_data_uri

Stored 'project_prefix' (str)
Stored 'bucket' (str)
Stored 'train_afd_tran_local_path' (str)
Stored 'test_afd_tran_local_path' (str)
Stored 's3_train_afd_tran_data_uri' (str)
Stored 's3_test_afd_tran_data_uri' (str)
